# Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [17]:
import requests

def max_rate():
    
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
    request_result = r.json()['Valute']

    valute_name = 'Валюта'
    max_rate = 0

    for key in request_result:
        if request_result[key]['Value'] > max_rate:
            max_rate = request_result[key]['Value']
            valute_name = request_result[key]['Name']
    
    print(f'Максимальное значение курса - у валюты {valute_name}: {max_rate}')

max_rate()

Максимальное значение курса - у валюты СДР (специальные права заимствования): 108.2375


# Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

### Вариант 1

In [48]:
class Rate:
    def __init__(self, format_='value'):
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self, diff=None):
        if diff is False or diff is None:
            return self.make_format('EUR')
        else: 
            response = self.exchange_rates()
            diff = response['EUR']['Value'] - response['EUR']['Previous']
            return diff
    
    def usd(self, diff=None):
        if diff is False or diff is None:
            return self.make_format('USD')
        else: 
            response = self.exchange_rates()
            diff = response['USD']['Value'] - response['USD']['Previous']
            return diff
    
    def brl(self):
        if diff is False or diff is None:
            return self.make_format('BRL')
        else: 
            response = self.exchange_rates()
            diff = response['BRL']['Value'] - response['BRL']['Previous']
            return diff

In [50]:
r = Rate()
r.eur(diff=True)

0.14109999999999445

### Вариант 2

In [29]:
import requests

class Rate:
    def __init__(self, format_='value', diff = False):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            
            if self.format == 'full':
                return response[currency]

            if self.format == 'value':
                if self.diff is None or self.diff is False:
                    return response[currency]['Value']
                
                if self.diff is True:
                    diff = response[currency]['Value'] - response[currency]['Previous']
                    return diff  
                      
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [33]:
r = Rate('value', diff = True).eur()
r

0.14109999999999445

# Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

### Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [75]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass
    
class Designer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def update_seniority(self, type_of_accreditation):
        if type_of_accreditation == 'regular':
            self.seniority += 1
        elif type_of_accreditation == 'international':
            self.seniority = self.seniority + 2
        
    def check_if_it_is_time_for_upgrade(self):
        if self.seniority % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [76]:
alex = Designer('Александр', 0)
alex.update_seniority(type_of_accreditation = 'regular')
alex.publish_grade()

Александр 1


In [79]:
alex.update_seniority(type_of_accreditation = 'international')
alex.update_seniority(type_of_accreditation = 'international')
alex.update_seniority(type_of_accreditation = 'international')
alex.check_if_it_is_time_for_upgrade()

Александр 2
